## **Load libraries and data**

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

In [ ]:
import pandas as pd
import numpy as np
import pythainlp
import gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [6]:
file = "1blcM1I2xm6fkvBVCHVxKze5khRne-2c8"
dwn_url=f"https://drive.google.com/uc?export=download&id={file}"
df = pd.read_csv(dwn_url)
df.head(3)

,Review ID,Review
0,1,เป็นคนที่ชอบทาน Macchiato เป็นประจำ มีวันนึงเด...
1,2,Art of Coffee Kasetsart เป็นร้านกาแฟรสชาติเยี่...
2,3,กวงทะเลเผา อาหารทะเลเค้าสดจริงๆเนื้อปูหวานไม่ค...


## **Tokenize Words**

In [ ]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')','ๆ','-','–','.',',','{','}',':',';','/','>','<','"']
screening_words = stopwords + removed_words

def tokenize_sentence(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      if merged == '':
        merged = word
      else:
        merged = merged + ',' + word
  return merged

In [ ]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_sentence(x))

In [ ]:
df.tail()

,Review ID,Review,Review_tokenized
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ..."
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย..."
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน..."
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค..."
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,..."


## **Create Dictionary**

In [ ]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

dict_keys(['20', 'Macchiato', 'กาแฟ', 'กาแฟร้อน', 'กิน', 'คน', 'ครึ่ง', 'ความคิด', 'ชอบ', 'ดื่ม', 'ตอน', 'ทาน', 'นึง', 'บาท', 'ปริมาณ', 'มีความสุข', 'ลา', 'สั่ง', 'หนัก', 'หลังจากนี้', 'เข้าไป', 'เจอ', 'เดิน', 'เดิม', 'เป็นประจำ', 'เฟล', 'แก้ว', 'โบราณ', 'Art', 'Coffee', 'Kasetsart', '^^', 'of', 'คาราเมล', 'คิ', 'ชา', 'ซ', 'ซี', 'ดี', 'นั่ง', 'น่ารัก', 'บรรยากาศ', 'พนักงาน', 'พูดจา', 'มัค', 'รสชาติ', 'ราคา', 'ร้านกาแฟ', 'ลอน', 'ลาเต้', 'ลูกค้า', 'อัธยาศัย', 'อัสสัม', 'อา', 'อิตาเลียน', 'อุดหนุน', 'เพรส', 'เมนู', 'เยี่ยม', 'เอ๊กซ์', 'แพง', 'โซดา', 'โซ่', 'โต้', 'ไอ', 'ไอซ์', '555', 'กก', 'กรอบ', 'กระ', 'กระทะ', 'กร๊อบ', 'กลิ่น', 'กวง', 'กะ', 'กินน้ำ', 'กุ้ง', 'ก้น', 'ข้าวผัด', 'ข้าวเหนียว', 'ความสนใจ', 'คอ', 'คาว', 'งั้น', 'งี้', 'จริ๊ง', 'จัดจ้าน', 'จาน', 'จี๊ดจ๊าด', 'จ้น', 'จ้ิม', 'จ๊อบ', 'ฉ่า', 'ชิม', 'ชุ่มคอ', 'ดาว', 'ตบท้าย', 'ตัก', 'ถูกใจ', 'ทอด', 'ทะเล', 'ทีหลัง', 'นานา', 'นุ่ม', 'น้ำ', 'น้ำปลา', 'บรรยาย', 'ปลา', 'ปลาหมึก', 'ปาก', 'ปู', 'ปูม้า', 'ผสม', 'ผัด', 'พง', 'พริก', 'พลาด'

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [ ]:
print(word_frequencies)

[[('20', 1), ('Macchiato', 2), ('กาแฟ', 1), ('กาแฟร้อน', 1), ('กิน', 1), ('คน', 1), ('ครึ่ง', 1), ('ความคิด', 1), ('ชอบ', 1), ('ดื่ม', 1), ('ตอน', 1), ('ทาน', 1), ('นึง', 1), ('บาท', 1), ('ปริมาณ', 1), ('มีความสุข', 1), ('ลา', 1), ('สั่ง', 1), ('หนัก', 1), ('หลังจากนี้', 1), ('เข้าไป', 1), ('เจอ', 1), ('เดิน', 1), ('เดิม', 1), ('เป็นประจำ', 1), ('เฟล', 2), ('แก้ว', 2), ('โบราณ', 1)], [('สั่ง', 1), ('Art', 1), ('Coffee', 1), ('Kasetsart', 1), ('^^', 1), ('of', 1), ('คาราเมล', 1), ('คิ', 1), ('ชา', 2), ('ซ', 1), ('ซี', 1), ('ดี', 1), ('นั่ง', 1), ('น่ารัก', 1), ('บรรยากาศ', 1), ('พนักงาน', 1), ('พูดจา', 1), ('มัค', 1), ('รสชาติ', 1), ('ราคา', 1), ('ร้านกาแฟ', 1), ('ลอน', 1), ('ลาเต้', 1), ('ลูกค้า', 1), ('อัธยาศัย', 1), ('อัสสัม', 1), ('อา', 1), ('อิตาเลียน', 1), ('อุดหนุน', 1), ('เพรส', 1), ('เมนู', 1), ('เยี่ยม', 1), ('เอ๊กซ์', 1), ('แพง', 1), ('โซดา', 1), ('โซ่', 1), ('โต้', 1), ('ไอ', 1), ('ไอซ์', 1)], [('กิน', 5), ('คน', 2), ('ชอบ', 1), ('ตอน', 1), ('ทาน', 3), ('นึง', 1), ('สั่ง', 3

## **Topic Modeling**

In [ ]:
num_topics = 10
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 7.23 s, sys: 75.5 ms, total: 7.3 s
Wall time: 7.26 s


In [ ]:
pyLDAvis.gensim_models.prepare(model, gensim_corpus, dictionary)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.049944  0.026142       1        1  31.331181
7     -0.044363  0.051520       2        1  13.311533
1     -0.057199 -0.032447       3        1  11.980251
3      0.017159 -0.017324       4        1   8.013889
2      0.016474 -0.037675       5        1   7.910945
0      0.001357  0.000185       6        1   7.392673
4      0.005670 -0.003736       7        1   6.362961
8      0.012618  0.006455       8        1   5.896440
6     -0.003651  0.005725       9        1   5.081133
5      0.001992  0.001154      10        1   2.718995, topic_info=         Term        Freq       Total Category  logprob  loglift
4         กิน  324.000000  324.000000  Default  30.0000  30.0000
11        ทาน  387.000000  387.000000  Default  29.0000  29.0000
907    ไข่มุก   80.000000   80.000000  Default  28.0000  28.0000
156     อร่อย  478.000000  478.000000  Default  27.0000  27.0000
35         ชา  234.000000  234.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
196       ใส่    3.171486  101.491446  Topic10  -5.6918   0.1391
147       หอม    3.375522  133.427369  Topic10  -5.6295  -0.0721
725      นะคะ    3.430967  146.469280  Topic10  -5.6132  -0.1491
39       นั่ง    3.153759  190.398153  Topic10  -5.6974  -0.4956
41   บรรยากาศ    2.976108  145.792227  Topic10  -5.7554  -0.2867

[787 rows x 6 columns], token_table=      Topic      Freq                                               Term
term                                                                    
3684      6  0.831210                                                   
4229      9  0.598549                                                "พร
4554      2  0.360239                                                 ++
4554      7  0.360239                                                 ++
318       1  0.259413                                                ...
...     ...       ...                                                ...
4643      2  0.835221                                                  •
4685      3  0.508133                                                 ••
3141      3  0.509828  …................................................
2122      4  0.548478                                                  ′
2123      4  0.547003                                                  ″

[1613 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 8, 2, 4, 3, 1, 5, 9, 7, 6])

In [ ]:
model.show_topic(3)

[('รสชาติ', 0.011885456),
 ('ทาน', 0.01160019),
 ('อร่อย', 0.011426737),
 ('กาแฟ', 0.009126503),
 ('สั่ง', 0.007809139),
 ('เมนู', 0.0077615487),
 ('กิน', 0.0074715656),
 ('หวาน', 0.006810424),
 ('ราคา', 0.0067505063),
 ('อาหาร', 0.0065948847)]

In [ ]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [ ]:
df[df['topics'] == 3]

,Review ID,Review,Review_tokenized,topics,score
4,5,ชอบมาทานร้านนี้ถ้าอยากกินอาหารเวียดนามใกล้บ้าน...,"ชอบ,ทาน,กิน,อาหาร,เวียดนาม,บ้าน,ซอย,ราชครู,เข้...",3,0.995255
60,61,ร้านก๋วยเตี๋ยว 22 เป็นร้านเก่าแก่ ครองใจคนชลบุ...,"ก๋วยเตี๋ยว,22,เก่าแก่,ครองใจ,คน,ชลบุรี,สิบ,ปี,...",3,0.997746
63,64,เข้าวังไปจิบกาแฟกันนะครับ บอกได้เลยว่า มาร้านน...,"วัง,จิบ,กาแฟ,อิ่ม,ท้อง,แล้วก็,อิ่มใจ,รู้เรื่อง...",3,0.997828
69,70,ร้านเชฟมด รอบนี้มาจัดเป็นรอบที่สอง หลังจากถูกใ...,"เชฟ,มด,รอบ,รอบ,ที่สอง,ถูกใจ,รอบ,เ,เร,ก,โรงเรีย...",3,0.999473
89,90,ผมเห็นรีวิวจากคุณ RiceMango ผมก็หาโอกาสว่างตาม...,"ผม,รีวิว,RiceMango,ผม,หา,โอกาส,ว่าง,ตามมา,ลอง,...",3,0.991728
99,100,อุดรมีร้านขายอาหารเช้าหลายร้าน แต่ละร้านก็จะมี...,"อุดร,ขาย,อาหารเช้า,เมนู,คล้าย ๆ,หนี,ไท่,พ้น,ไข...",3,0.992900
189,190,ร้าน ไออุ่น ชาไข่มุก มีนํ้าปั่นหลากหลายรสชาติ ...,"ไออุ่น,ชา,ไข่มุก,นํ้า,ปั่น,หลากหลาย,รสชาติ,ลิ้...",3,0.991494
193,194,ห้องแอร์ กว้างขวาง สมราคารสชาติเข้มข้น พอดีดื่...,"ห้อง,แอร์,สมราคา,รสชาติ,เข้มข้น,ดื่ม,อ,เม,กา,โ...",3,0.996259
210,211,ถ้าอยากกินข้าวห่อไข่จะนึกถึงร้านนี้เป็นร้านแรก...,"กิน,ข้าวห่อ,ไข่,นึกถึง,เมนู,เลือก,หลากหลาย,ข้า...",3,0.990983
212,213,แดดเริ่มร่ม ลมเริ่มตกก็ถึงเพลานั่งจิบชากาแฟตาม...,"แดด,ร่ม,ลม,ตก,เพลา,นั่ง,จิบ,ชา,กาแฟ,ธรรมเนียม,...",3,0.998225
